# Python Insights - Analisando Dados com Python

### Case - Cancelamento de Clientes

Você foi contratado por uma empresa com mais de 800 mil clientes para um projeto de Dados. Recentemente a empresa percebeu que da sua base total de clientes, a maioria são clientes inativos, ou seja, que já cancelaram o serviço.

Precisando melhorar seus resultados ela quer conseguir entender os principais motivos desses cancelamentos e quais as ações mais eficientes para reduzir esse número.

Base de dados e arquivos: https://drive.google.com/drive/folders/1uDesZePdkhiraJmiyeZ-w5tfc8XsNYFZ?usp=drive_link

In [3]:
import pandas as pd

tabela = pd.read_csv("matches.csv")

tabela = tabela.drop(columns="Équipe Domicile")
tabela = tabela.drop(columns="Équipe Extérieur")
tabela = tabela.drop(columns="Saison")
tabela = tabela.drop(columns="Date")
tabela = tabela.drop(columns="Heure")
tabela = tabela.drop(columns="Score Domicile")
tabela = tabela.drop(columns="Score Extérieur")
display(tabela)

,Championat,Cote 1,Cote X,Cote 2,Gangnant
0,france/ligue-1,1.45,3.80,5.60,1.0
1,france/ligue-1,2.94,2.61,2.45,2.0
2,france/ligue-1,1.95,3.50,2.95,1.0
3,france/ligue-1,1.47,3.43,6.20,1.0
4,france/ligue-1,2.24,3.45,2.49,1.0
...,...,...,...,...,...
27593,switzerland/super-league,2.63,3.38,2.58,0.0
27594,switzerland/super-league,2.52,3.40,2.70,1.0
27595,switzerland/super-league,2.05,3.78,3.23,1.0
27596,switzerland/super-league,2.18,3.52,3.16,1.0


In [4]:
display(tabela.info())

tabela = tabela.dropna()

display(tabela.info())
tabela.to_csv("matches_tratada.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Championat  27598 non-null  object 
 1   Cote 1      27479 non-null  float64
 2   Cote X      27479 non-null  float64
 3   Cote 2      27479 non-null  float64
 4   Gangnant    27423 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.1+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 27423 entries, 0 to 27597
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Championat  27423 non-null  object 
 1   Cote 1      27423 non-null  float64
 2   Cote X      27423 non-null  float64
 3   Cote 2      27423 non-null  float64
 4   Gangnant    27423 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.3+ MB


None

In [ ]:
display(tabela["Gangnant"].value_counts())

display(tabela["Gangnant"].value_counts(normalize=True))

display(tabela["Gangnant"].value_counts(normalize=True).map("{:.1%}".format))

Gangnant
1.0    12603
2.0     7794
0.0     7026
Name: count, dtype: int64

Gangnant
1.0    0.459578
2.0    0.284214
0.0    0.256208
Name: proportion, dtype: float64

Gangnant
1.0    46.0%
2.0    28.4%
0.0    25.6%
Name: proportion, dtype: object

In [16]:
import plotly.express as px
empates_por_campeonato = tabela[tabela["Gangnant"] == 0].groupby("Championat").size() / tabela.groupby("Championat").size()
empates_por_campeonato = empates_por_campeonato.reset_index(name="Porcentagem de Empates")  # Reset para formatação
grafico_empates = px.bar(
    empates_por_campeonato, 
    x="Championat", 
    y="Porcentagem de Empates",
    title="Porcentagem de Empates por Campeonato",
    labels={"Porcentagem de Empates": "Porcentagem", "Championat": "Campeonato"},
)

grafico_empates.show()

In [ ]:
def maior_probabilidade(row):
    if row["Probabilidade Mandante (%)"] > max(row["Probabilidade Visitante (%)"], row["Probabilidade Empate (%)"]):
        return "Mandante"
    elif row["Probabilidade Visitante (%)"] > max(row["Probabilidade Mandante (%)"], row["Probabilidade Empate (%)"]):
        return "Visitante"
    else:
        return "Empate"

tabela["Maior Probabilidade"] = tabela.apply(maior_probabilidade, axis=1)

resultado_map = {1: "Mandante", 2: "Visitante", 0: "Empate"}
tabela["Resultado Real"] = tabela["Gangnant"].map(resultado_map)

import plotly.express as px

grafico_comparacao = px.histogram(
    tabela,
    x="Maior Probabilidade",
    color="Resultado Real",
    barmode="group",
    title="Comparação: Maior Probabilidade vs Resultado Real",
    labels={
        "Maior Probabilidade": "Maior Probabilidade (Prevista)",
        "Resultado Real": "Resultado Real"
    }
)
tabela["Acerto"] = tabela["Maior Probabilidade"] == tabela["Resultado Real"]
porcentagem_acerto = tabela["Acerto"].mean() * 100
erros = tabela[tabela["Acerto"] == False]

erros["Erro"] = erros["Maior Probabilidade"] + " → " + erros["Resultado Real"]

media_erros = erros["Erro"].value_counts(normalize=True) * 100

print("Distribuição dos erros (em porcentagem):")
print(media_erros.round(2))
print(f"Porcentagem de acerto da maior probabilidade com o resultado real: {porcentagem_acerto:.2f}%")
grafico_comparacao.show()


Distribuição dos erros (em porcentagem):
Erro
Mandante → Empate       38.79
Mandante → Visitante    29.53
Visitante → Empate      15.85
Visitante → Mandante    15.21
Empate → Mandante        0.39
Empate → Visitante       0.24
Name: proportion, dtype: float64
Porcentagem de acerto da maior probabilidade com o resultado real: 53.58%


C:\Users\ianni\AppData\Local\Temp\ipykernel_17684\1636419173.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def verifica_acerto(row):
    menor_odd = min(row["Cote 1"], row["Cote X"], row["Cote 2"])
    if menor_odd == row["Cote 1"] and row["Gangnant"] == 1:
        return True
    elif menor_odd == row["Cote 2"] and row["Gangnant"] == 2:
        return True
    elif menor_odd == row["Cote X"] and row["Gangnant"] == 0:
        return True
    return False

tabela["Acerto"] = tabela.apply(verifica_acerto, axis=1)
acertos_por_campeonato = tabela.groupby("Championat")["Acerto"].mean().reset_index(name="Taxa de Acerto")

grafico_acertos = px.bar(
    acertos_por_campeonato,
    x="Championat",
    y="Taxa de Acerto",
    title="Taxa de Acerto por Campeonato (Menor Odd)",
    labels={"Taxa de Acerto": "Porcentagem de Acertos", "Championat": "Campeonato"},
)

grafico_acertos.show()

In [ ]:
resultados_distribuicao = tabela["Gangnant"].value_counts(normalize=True).reset_index()
resultados_distribuicao.columns = ["Resultado", "Proporção"]
resultados_distribuicao["Resultado"] = resultados_distribuicao["Resultado"].map({1: "Mandante", 2: "Visitante", 0: "Empate"})

grafico_resultados = px.pie(
    resultados_distribuicao,
    names="Resultado",
    values="Proporção",
    title="Distribuição de Resultados: Mandante, Visitante e Empate"
)

grafico_resultados.show()

In [ ]:
resultados_distribuicao = (
    tabela.groupby(["Championat", "Gangnant"])
    .size()
    .reset_index(name="Quantidade")
)

resultados_distribuicao["Proporção"] = (
    resultados_distribuicao["Quantidade"]
    / resultados_distribuicao.groupby(["Championat"])["Quantidade"].transform("sum")
)

resultados_distribuicao["Resultado"] = resultados_distribuicao["Gangnant"].map(
    {1: "Mandante", 2: "Visitante", 0: "Empate"}
)

cores_resultados = {
    "Mandante": "blue",
    "Visitante": "red",
    "Empate": "green"
}

for campeonato, dados in resultados_distribuicao.groupby("Championat"):
    grafico = px.pie(
        dados,
        names="Resultado",
        values="Proporção",
        title=f"Distribuição de Resultados: {campeonato}",
        color="Resultado", 
        color_discrete_map=cores_resultados, 
    )
    grafico.show()

In [ ]:
relacao_odds_resultados = tabela.groupby("Gangnant")[["Cote 1", "Cote X", "Cote 2"]].mean().reset_index()

relacao_odds_resultados["Resultado"] = relacao_odds_resultados["Gangnant"].map({1: "Mandante", 2: "Visitante", 0: "Empate"})

relacao_odds_resultados = relacao_odds_resultados.drop(columns=["Gangnant"])

print(relacao_odds_resultados)  


     Cote 1    Cote X    Cote 2  Resultado
0  2.557859  3.591003  4.013890     Empate
1  2.064204  4.065300  5.762274   Mandante
2  3.579724  3.722628  3.095606  Visitante


In [ ]:
def verifica_acerto(row):
    menor_odd = min(row["Cote 1"], row["Cote X"], row["Cote 2"])
    if menor_odd == row["Cote 1"] and row["Gangnant"] == 1:
        return True
    elif menor_odd == row["Cote 2"] and row["Gangnant"] == 2:
        return True
    elif menor_odd == row["Cote X"] and row["Gangnant"] == 0:
        return True
    return False

tabela["Acerto"] = tabela.apply(verifica_acerto, axis=1)

acertos_por_campeonato = tabela.groupby("Championat")["Acerto"].mean().reset_index()
acertos_por_campeonato.columns = ["Campeonato", "Taxa de Acerto"]

maior_assertividade = acertos_por_campeonato.loc[acertos_por_campeonato["Taxa de Acerto"].idxmax()]
menor_assertividade = acertos_por_campeonato.loc[acertos_por_campeonato["Taxa de Acerto"].idxmin()]

print("Campeonato com maior assertividade:")
print((maior_assertividade*100))

print("\nCampeonato com menor assertividade:")
print(menor_assertividade*100)

Campeonato com maior assertividade:
Campeonato        aruba/division-di-honoraruba/division-di-honor...
Taxa de Acerto                                            67.942584
Name: 5, dtype: object

Campeonato com menor assertividade:
Campeonato        albania/abissnet-superiorealbania/abissnet-sup...
Taxa de Acerto                                            36.666667
Name: 0, dtype: object


In [ ]:
desempenho_mandante_visitante = tabela["Gangnant"].value_counts().reset_index()
desempenho_mandante_visitante.columns = ["Resultado", "Quantidade"]

desempenho_mandante_visitante["Resultado"] = desempenho_mandante_visitante["Resultado"].map({1: "Mandante", 2: "Visitante", 0: "Empate"})

print(desempenho_mandante_visitante)

   Resultado  Quantidade
0   Mandante       12603
1  Visitante        7794
2     Empate        7026


In [ ]:
porcentagem_empates_por_campeonato = (
    tabela[tabela["Gangnant"] == 0].groupby("Championat").size() / tabela.groupby("Championat").size()
).reset_index(name="Porcentagem de Empates")

campeonato_maior_porcentagem_empates = porcentagem_empates_por_campeonato.loc[
    porcentagem_empates_por_campeonato["Porcentagem de Empates"].idxmax()
]
campeonato_menor_porcentagem_empates = porcentagem_empates_por_campeonato.loc[
    porcentagem_empates_por_campeonato["Porcentagem de Empates"].idxmin()
]
print("Campeonato com maior porcentagem de empates:")
print(campeonato_maior_porcentagem_empates*100)
print("Campeonato com menor porcentagem de empates:")
print(campeonato_menor_porcentagem_empates*100)


Campeonato com maior porcentagem de empates:
Championat                argentina/torneo-betanoargentina/torneo-betano...
Porcentagem de Empates                                            41.176471
Name: 3, dtype: object
Campeonato com menor porcentagem de empates:
Championat                aruba/division-di-honoraruba/division-di-honor...
Porcentagem de Empates                                            11.004785
Name: 5, dtype: object
